In [22]:
import pandas as pd 

total_data = pd.read_csv("winter_wheat_yield.csv")




weather_params = ['T2M_MAX','T2M_MIN', 'PRECTOTCORR', 'GWETROOT', 'EVPTRNS', 'ALLSKY_SFC_PAR_TOT', 'HDD10', 'SNODP']
'''
   T2M_MAX: The maximum hourly air (dry bulb) temperature at 2 meters above the surface of the 
             earth in the period of interest.
   T2M_MIN: The minimum hourly air (dry bulb) temperature at 2 meters above the surface of the 
            earth in the period of interest.
   PRECTOTCORR: The bias corrected average of total precipitation at the surface of the earth 
                in water mass (includes water content in snow)
   EVPTRNS: The evapotranspiration energy flux at the surface of the earth
   ALLSKY_SFC_PAR_TOT: The total Photosynthetically Active Radiation (PAR) incident 
         on a horizontal plane at the surface of the earth under all sky conditions
    HDD10: The daily accumulation of degrees when the daily mean temperature is below 10 degrees Celsius.
    (https://extension.sdstate.edu/what-makes-winter-wheat-winter-wheat)
    SNODP: The snow depth on land at surface of the earth. (https://extension.sdstate.edu/effects-snow-wheat)
'''



base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?"
base_url += 'parameters=T2M_MAX,T2M_MIN,PRECTOTCORR,GWETROOT,EVPTRNS,ALLSKY_SFC_PAR_TOT,HDD10,SNODP&'
base_url += 'community=RE&longitude={longitude}&latitude={latitude}&start={prev_year}1001&end={year}0430&format=JSON'

#base_url2 = r"https://power.larc.nasa.gov/api/temporal/daily/point?"
#base_url2 += 'parameters=T2M_MAX,T2M_MIN,PRECTOTCORR,GWETROOT,EVPTRNS,ALLSKY_SFC_PAR_TOT,HDD10,SNODP&'
#base_url2 += 'community=RE&longitude={longitude}&latitude={latitude}&start={year}0101&end={year}0430&format=JSON'


import json
import requests

def fetch_weather_county_year(year, state, county):
    row = total_data.loc[(total_data['state_name'] == state) & \
                        (total_data['county_name'] == county) & \
                        (total_data['year'] == year)]    
    print(row)
    # print(type(row))
    lon = row.iloc[0]['lon']
    lat = row.iloc[0]['lat']
    # print(lon, lat)
    
    api_request_url = base_url.format(longitude=lon, latitude=lat, prev_year=str(year - 1), year=str(year))

    #api_request_url2 = base_url2.format(longitude=lon, latitude=lat, year=str(year))
    # this api request returns a json file
    response = requests.get(url=api_request_url, verify=True, timeout=30.00)
    #response2 = requests.get(url=api_request_url2, verify=True, timeout=30.00)
    # print(response.status_code)
    content = json.loads(response.content.decode('utf-8'))
    #content2 = json.loads(response2.content.decode('utf-8'))

    # print('\nType of content object is: ', type(content))
    # print(json.dumps(content, indent=4))

    # print('\nKeys of content dictionary are: \n', content.keys())
    # print('\nKeys of "properties" sub-dictionary are: \n', content['properties'].keys())
    # print('\nKeys of "parameter" sub-dictionary are: \n', content['properties']['parameter'].keys())
    # print()
    weather = content['properties']['parameter']
    #weather2 = content2['properties']['parameter']

    df = pd.DataFrame(weather)
    #df_2 = pd.DataFrame(weather2)
    #merged_df = pd.concat([df_2, df])
    
    #return merged_df
    return df

# sanity check
df = fetch_weather_county_year(2022, 'INDIANA', 'TIPPECANOE')

# examining the output
print(len(df))
print()
print(df.tail())

      year state_name county_name  Value        lon        lat
3090  2022    INDIANA  TIPPECANOE   85.8 -86.887969  40.391607
212

          T2M_MAX  T2M_MIN  PRECTOTCORR  GWETROOT  EVPTRNS  \
20220426    11.79     0.83         0.00      0.80     0.17   
20220427    13.13    -0.90         0.00      0.80     0.21   
20220428    15.80     3.50         0.18      0.80     0.16   
20220429    19.08     7.64         0.12      0.79     0.30   
20220430    23.71    10.87         7.81      0.79     0.32   

          ALLSKY_SFC_PAR_TOT  HDD10  SNODP  
20220426              106.34   3.69    0.0  
20220427              123.98   3.88    0.0  
20220428               75.12   0.38    0.0  
20220429               81.56   0.00    0.0  
20220430               75.43   0.00    0.0  


In [24]:
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

# w_df will be a dictionary of df's, each holding weather info for
#    one year-state-county triple
# The dictionary keys will be the index values of df_yscyll that we
#    built above (also archived in year_state_county_yield_lon_lat.csv)
w_df = {}

# will archive each w_df[i] value into a csv as we go along, for 2 reasons:
#   - because this takes forever to run
#   - network connectivity or other errors in middle of run may abort the process
out_dir = 'WEATHER-DATA--v01/'
filename = r'weather-data-for-index__{index}.csv'

starttime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")


for i in range(785, len(total_data)):
    row = total_data.iloc[i]
    # print(row)
    w_df[i] = fetch_weather_county_year(row['year'], row['state_name'], row['county_name'])
    outfilename = out_dir + filename.format(index=str(i).zfill(4))
    w_df[i].to_csv(outfilename)
    # adding this to get a feeling of forward progress
    if i % 10 == 0:
        print('\nFinished work on index: ', i, '     at time: ', datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('   This involved fetching weather data for the following row:')
        print(row['year'],row['state_name'],row['county_name'], row['lon'],row['lat'])
        print('Wrote file: ', outfilename)

'''
# sanity check
print()
index = 2
print(r'The contents of yscyll for index {index} is:'.format(index=index), '\n')
print(df_yscyll.iloc[index])
print()
print(r'The head of the weather data for index {index} is:'.format(index=index), '\n')
print(w_df[index].head(10))
'''

endtime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print('start and end times were: ', starttime, endtime)

     year  state_name county_name  Value         lon        lat
785  2014  CALIFORNIA  SACRAMENTO   50.0 -121.493895  38.581061
     year  state_name county_name  Value         lon        lat
786  2011  CALIFORNIA  SACRAMENTO   89.5 -121.493895  38.581061
     year  state_name county_name  Value         lon        lat
787  2008  CALIFORNIA  SACRAMENTO   89.0 -121.493895  38.581061
     year  state_name county_name  Value         lon        lat
788  2006  CALIFORNIA  SACRAMENTO   52.0 -121.493895  38.581061
     year  state_name county_name  Value         lon        lat
789  2005  CALIFORNIA  SACRAMENTO   67.1 -121.493895  38.581061
     year  state_name county_name  Value         lon        lat
790  2004  CALIFORNIA  SACRAMENTO   80.0 -121.493895  38.581061

Finished work on index:  790      at time:  2023-06-04 15:13:52
   This involved fetching weather data for the following row:
2004 CALIFORNIA SACRAMENTO -121.493895 38.5810606
Wrote file:  WEATHER-DATA--v01/weather-data-for-index__

KeyboardInterrupt: 